In [253]:
'''
In this work the movie plots loaded along with their titles, and the female lead character has been recognized
and labeled as Female Protagonist 

'''
#Importing liberaries
import pandas as pd
import re

import nltk
from nltk import pos_tag
import spacy
nlp = spacy.load("en_core_web_sm")  #loading the built in package 
from collections import Counter

In [70]:
#!pip install spacy

In [71]:
#!python -m spacy download en_core_web_sm

In [254]:
#reading files 
#Taking the first 100 movie plots

with open('plots', 'r', encoding='utf-8') as f:
    movie_summary = f.read()

with open('titles', 'r', encoding='utf-8') as t:
    movie_title = t.read()


movie_summaries = movie_summary[0:600000].split('<EOS>')[0:100]

In [255]:
#Taking first 100 movie plots
titles = movie_title.split('\n')[0:100]

In [256]:
d = {'Movie_plots': movie_summaries, 'Titles': titles} #making dictionary from movie plots and titles
df = pd.DataFrame(data = d)  #making dataframe from 100 movie plots and respective movie titles

In [257]:
df.head()

,Movie_plots,Titles
0,"Old Major, the old boar on the Manor Farm, sum...",Animal Farm
1,\nAlex is a 15-year-old living in near-future ...,A Clockwork Orange (novel)
2,\nThe text of The Plague is divided into five ...,The Plague
3,"\nAmong others, John Heath has observed, ""The ...",Actaeon
4,"\nAn expedition from Straumli Realm, an ambiti...",A Fire Upon the Deep


In [262]:
def spacy_pronoun_freq(string):
    
    '''
    function returns the most frequent pronoun came in the movie plot
    first cleans the string or sentence using regular expressions, 
    then it makes a list of pronouns based on the movie plot and using Counter method the most common pronoun is returned
    
    '''
    
    string =  re.sub(r'[^a-zA-Z0-9\s]', '', string)    #removing every other characters or symbols except 
                                                       #whats in the squre bracket 
    string = re.sub('\n', '', string)   

    pronoun_list =[]
    doc = nlp(string)  #using the spacy object accessing the strings and tagging it with 
                        #respective part of speech (pos tagging)
    for token in doc:
        if token.tag_ == 'PRP':            #appending all the tokens(pronouns) in the pronoun_list using token.tag_ 
            pronoun_list.append(str(token))     #where PRP stands for pronoun
            
            
    if Counter(pronoun_list):     
        return Counter(pronoun_list).most_common(1)[0]   #returning the most frequently used pronoun to find out 
                                                        #which gender is more predominant

In [ ]:
'''
#here the function finds out the main character name using spacy pos tagging

def spacy_noun_freq(string):
  
    string =  re.sub(r'[^a-zA-Z0-9\s]', '', string)   #removing puntuations or newline from the string
    string = re.sub('\n', '', string)

    noun_list =[]
    doc = nlp(string)
    for token in doc:
        if token.tag_ == 'NNP':  #NNP stands for nominative noun and helps append all the noun name mentioned in the sentence
            noun_list.append(str(token))    

    if Counter(noun_list):
        return Counter(noun_list).most_common(1)[0]   #counting frequency of most common noun name mentioned in the sentence
                                                    #and returning which has been mentioned mostly
'''

In [264]:
def finding_female_protagonist(df, plots):
    frequent_pronoun = []      #making an empty list
    frequent_pronoun = df[plots].apply(spacy_pronoun_freq) #mapping spacy_pronoun_freq function to eac row of the dataframe
    for i in range(len(frequent_pronoun)):
        try:
            if frequent_pronoun[i][0] == 'she':    #based on the most frequent pronoun 'she' labelling that 
                df.loc[i, 'Protagonist']= 'Female Protagonist'       #row as 'female protagonist'

            else:
                df.loc[i, 'Protagonist']='other'   #non female protagonist are labeled as others
        except:
            pass
    return df            #returning dataframe with additional column about the lead character (Protagonist)

In [265]:
df2 = finding_female_protagonist(df, 'Movie_plots')  #the function will return a dataframe with labeled movie plots

In [267]:
df2.head(10)

,Movie_plots,Titles,Protagonist
0,"Old Major, the old boar on the Manor Farm, sum...",Animal Farm,other
1,\nAlex is a 15-year-old living in near-future ...,A Clockwork Orange (novel),other
2,\nThe text of The Plague is divided into five ...,The Plague,other
3,"\nAmong others, John Heath has observed, ""The ...",Actaeon,other
4,"\nAn expedition from Straumli Realm, an ambiti...",A Fire Upon the Deep,other
5,"\nThe book tells the story of Paul Bäumer, a G...",All Quiet on the Western Front,other
6,\nThe story is set in an imaginary American to...,Anyone Can Whistle,Female Protagonist
7,"\nIn ancient Rome, some neighbors live in thre...",A Funny Thing Happened on the Way to the Forum,other
8,"\nBeing transported to the Middle Ages, Ash Wi...",Army of Darkness,other
9,\nThe film follows two juxtaposed families.\nO...,The Birth of a Nation,other


In [266]:
df2[df2['Protagonist']=='Female Protagonist']

,Movie_plots,Titles,Protagonist
6,\nThe story is set in an imaginary American to...,Anyone Can Whistle,Female Protagonist
15,\nHigh school senior Buffy Summers (Kristy Swa...,Buffy the Vampire Slayer (film),Female Protagonist
22,"\nIn Gotham, one year after the defeat of Two ...",Batman &amp; Robin (film),Female Protagonist
36,\nLe Fanu presents the story as part of the ca...,Carmilla,Female Protagonist
68,"\nIn 2065, Earth is infested by alien life for...",Final Fantasy: The Spirits Within,Female Protagonist
70,"\nTwo girls, Agnes and Elin, attend school in ...",Show Me Love (film),Female Protagonist
84,\nWhen the Japanese freighter Eiko-maru is des...,Godzilla (1954 film),Female Protagonist
86,\nHarriet Vane returns with trepidation to Shr...,Gaudy Night,Female Protagonist
91,\nGone with the Wind takes place in the southe...,Gone with the Wind (novel),Female Protagonist
95,\nHarold Chasen (Bud Cort) is a young man obse...,Harold and Maude,Female Protagonist


In [104]:
'''
#In the similar way nltk liberary can be to find out the most frequent noun and pronoun and the movie plots can be labeled
#Although its quite slow as compare to spacy's pos tagging


def nltk_noun_counting(string):
    string = re.sub('[^\sa-zA-Z0-9]', '', string )
    string = re.sub('\n', '', string)
    string = string.split()
    noun_list = []
    for i in range(len(string)):
        tags = nltk.pos_tag(string)
        if tags[i][1]=='NNP':
            noun_list.append(tags[i][0])
    freq_noun = Counter(noun_list).most_common(1)
    
    return freq_noun


def nltk_pronoun_counting(string):
    string = re.sub('[^\sa-zA-Z0-9]', '', string )
    string = re.sub('\n', '', string)
    string = string.split()
    pronoun_list = []
    for i in range(len(string)):
        tags = nltk.pos_tag(string)
        if tags[i][1]=='PRP':
            pronoun_list.append(tags[i][0])
    freq_pronoun = tuple(Counter(pronoun_list).most_common(1))
    return freq_pronoun
    
    
#df['Movies_noun_counts'] = df['Movies_plots'][:5].apply(nltk_pronoun_counting)
'''